In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4398k      0  0:00:18  0:00:17  0:00:01 5510k2M   69 55.4M    0     0  4094k      0  0:00:20  0:00:13  0:00:07 5411k    0  4444k      0  0:00:18  0:00:18 --:--:-- 5495k


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
from utils import create_dataloaders
from torch import optim

In [2]:
#from transformer import Transformer

In [3]:
train_dir = "aclImdb/train"
train_dataloader, vocab = create_dataloaders(train_dir)

In [4]:
embed_dim = 256
num_heads = 2
dense_dim = 32

In [5]:
import torch.nn as nn
from transformer_encoder import TransformerEncoder
import torch

class Transformer(nn.Module):
    def __init__(self, embed_dim, dense_dim, num_heads, vocab_size, **kwargs):
        super(Transformer, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)


        self.encoder = TransformerEncoder(embed_dim, dense_dim, num_heads)
        # self.decoder = 

        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(600, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, text, mask=None):
        embedded = self.embedding(text)
        encoder_output = self.encoder(embedded, mask)
        # decoder_output = self.decoder(encoder_output, mask)
        output = self.global_max_pool(encoder_output)
        output = output.squeeze(-1)  # (batch_size, embed_dim) -> (batch_size, 1)
        output = self.out(output)
        output = self.sigmoid(output)
        return output


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transformer = Transformer(embed_dim, dense_dim, num_heads, 20000).to(device)

rmsprop = optim.RMSprop(params=transformer.parameters())
criterion = nn.NLLLoss()


for epoch in range(10):
    for batch in train_dataloader:
        text, label = batch

        rmsprop.zero_grad()

        output = transformer(text.to(device))

        loss = criterion(output[:, 0], label.to(device))

        print(output, label)

        loss.backward()
        rmsprop.step()
        break
 
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


Epoch: 1, Loss: -1.0
Epoch: 2, Loss: -1.0
Epoch: 3, Loss: -1.0
Epoch: 4, Loss: -1.0
Epoch: 5, Loss: -1.0
Epoch: 6, Loss: -1.0
Epoch: 7, Loss: -1.0
Epoch: 8, Loss: -1.0
Epoch: 9, Loss: -1.0
Epoch: 10, Loss: -1.0
